# Project Machine Learning Kelompok 4
**Tahapan Exploratory Data Analysis**

Nama Kelompok :

1. Wilhelmus Medhavi
2. Alfiyanti S.
3. Mayang Indi G
4. Sumayya
5. Anggina Dwi F.
6. Credenda M.
7. Dimas Fauzi Prasetyo

In [3]:
# Import modul
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn import under_sampling, over_sampling
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import chi2_contingency

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# Load dataset
train = pd.read_csv('/content/drive/MyDrive/Source Code/Training Data.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Id                 252000 non-null  int64 
 1   Income             252000 non-null  int64 
 2   Age                252000 non-null  int64 
 3   Experience         252000 non-null  int64 
 4   Married/Single     252000 non-null  object
 5   House_Ownership    252000 non-null  object
 6   Car_Ownership      252000 non-null  object
 7   Profession         252000 non-null  object
 8   CITY               252000 non-null  object
 9   STATE              252000 non-null  object
 10  CURRENT_JOB_YRS    252000 non-null  int64 
 11  CURRENT_HOUSE_YRS  252000 non-null  int64 
 12  Risk_Flag          252000 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 25.0+ MB


# New Section

**Data Cleansing**

Membersihkan data pada kolom CITY dan STATE, menghilangkan value yang memiliki ['angka'] dan [ ] 

In [6]:
train['STATE'].unique()

array(['Madhya_Pradesh', 'Maharashtra', 'Kerala', 'Odisha', 'Tamil_Nadu',
       'Gujarat', 'Rajasthan', 'Telangana', 'Bihar', 'Andhra_Pradesh',
       'West_Bengal', 'Haryana', 'Puducherry', 'Karnataka',
       'Uttar_Pradesh', 'Himachal_Pradesh', 'Punjab', 'Tripura',
       'Uttarakhand', 'Jharkhand', 'Mizoram', 'Assam',
       'Jammu_and_Kashmir', 'Delhi', 'Chhattisgarh', 'Chandigarh',
       'Uttar_Pradesh[5]', 'Manipur', 'Sikkim'], dtype=object)

In [7]:
# Kolom yang dibersihkan adalah kota dan negara bagian
# Membuat list untuk menyimpan data yang dibersihkan
list_city = train['CITY']
list_state = train['STATE']
list_new_city = []
list_new_state = []

# Eksekusi pembersihan data
for i in list_city :
  city = i.split('[')[0].strip()
  list_new_city.append(city)
for i in list_state :
  state = i.split('[')[0].strip()
  list_new_state.append(state)

# Memasukan kolom ke dataset
train['City'] = list_new_city
train['State'] = list_new_state

In [8]:
# Membersihkan data state & city yang aneh

# delete strage value in state column
train['STATE'] = train['STATE'].str.replace('\d+','')
train['STATE'] = train['STATE'].str.replace('[','')
train['STATE'] = train['STATE'].str.replace(']','')

# delete strage value in city column
train['CITY'] = train['CITY'].str.replace('\d+','')
train['CITY'] = train['CITY'].str.replace('[','')
train['CITY'] = train['CITY'].str.replace(']','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWar

A. Handle missing values

In [9]:
# Mengecek nilai null
train.isna().sum()

Id                   0
Income               0
Age                  0
Experience           0
Married/Single       0
House_Ownership      0
Car_Ownership        0
Profession           0
CITY                 0
STATE                0
CURRENT_JOB_YRS      0
CURRENT_HOUSE_YRS    0
Risk_Flag            0
City                 0
State                0
dtype: int64

Berdasarkan hasil pengecekan tidak terdapat missing values.

B. Handle duplicated data

In [10]:
# Mengecek data duplikat
train.duplicated().sum()

0

In [11]:
train['Id'].duplicated().sum()

0

Berdasarkan hasil pengecekan tidak terdapat duplicated data values.

C. Handle outliers

In [12]:
# Mengecek outlier
# Pada tahap EDA tidak terdapat outlier dari boxplot, sekarang menggunakan z-score

kontinu = ['Id','Income','Age','Experience','CURRENT_JOB_YRS','CURRENT_HOUSE_YRS','Risk_Flag']
print(f'Jumlah baris sebelum memfilter outlier: {len(train)}')
filtered_entries = np.array([True] * len(train))

for col in kontinu:
    zscore = abs(stats.zscore(train[col]))
    filtered_entries = (zscore < 3)
    
train_fil = train[filtered_entries]
print(f'Jumlah baris setelah memfilter outlier: {len(train_fil)}')

Jumlah baris sebelum memfilter outlier: 252000
Jumlah baris setelah memfilter outlier: 252000


Berdasarkan hasil pengecekan tidak terdapat outliers, hal ini juga terlihat dari visualisasi data pada tahap EDA.

D. Feature Transformation

In [13]:
# Transformasi / normalisasi Data dengan minmaxscaler pada kolom data kontinu exp, age, income, dan current house year
# Tujuannya adalah untuk mendapatkan data dengan skala yang sama

train['Experience_norm'] = MinMaxScaler().fit_transform(train['Experience'].values.reshape(len(train), 1))
train['Age_norm'] = MinMaxScaler().fit_transform(train['Age'].values.reshape(len(train), 1))
train['Income_norm'] = MinMaxScaler().fit_transform(train['Income'].values.reshape(len(train), 1))
train['House_norm'] = MinMaxScaler().fit_transform(train['CURRENT_HOUSE_YRS'].values.reshape(len(train), 1))
train.describe()

,Id,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Experience_norm,Age_norm,Income_norm,House_norm
count,252000.000000,2.520000e+05,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000
mean,126000.500000,4.997117e+06,49.954071,10.084437,6.333877,11.997794,0.123000,0.504222,0.499208,0.499198,0.499448
std,72746.278255,2.878311e+06,17.063855,6.002590,3.647053,1.399037,0.328438,0.300129,0.294204,0.288130,0.349759
min,1.000000,1.031000e+04,21.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,63000.750000,2.503015e+06,35.000000,5.000000,3.000000,11.000000,0.000000,0.250000,0.241379,0.249529,0.250000
50%,126000.500000,5.000694e+06,50.000000,10.000000,6.000000,12.000000,0.000000,0.500000,0.500000,0.499557,0.500000
75%,189000.250000,7.477502e+06,65.000000,15.000000,9.000000,13.000000,0.000000,0.750000,0.758621,0.747495,0.750000
max,252000.000000,9.999938e+06,79.000000,20.000000,14.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Berdasarkan hasil hasil normalisasi, jangkauan nilai dari ketiga variabel (age, income, dan experience) menjadi 0-1. Hal ini dilakukan untuk memudahkan mesin dalam melakukan permodelan. 

E. Feature Encoding

In [14]:
# Melakukan encoding pada data-data string kategori
 
mapping_married_single = {
    'single' : 0,
    'married' : 1
}

mapping_house = {
    'norent_noown' : 0,
    'rented' : 1,
    'owned' : 2
}

mapping_car = {
    'no' : 0,
    'yes' : 1
}

train['Married/Single'] = train['Married/Single'].map(mapping_married_single)
train['House_Ownership'] = train['House_Ownership'].map(mapping_house)
train['Car_Ownership'] = train['Car_Ownership'].map(mapping_car)

In [15]:
# Perlu dilihat urutan rasio customer yg gagal membayar untuk mempermudah encoding pada kolom profesi, kota, dan negara bagian 
# mempersiapkan dataset untuk customer yang gagal dan tidak

train_nr = train[(train['Risk_Flag']==0)]
train_r = train[(train['Risk_Flag']==1)]

In [16]:
# Mengecek urutan persentase rasio customer yang gagal membayar berdasarkan kota
city_NR = train_nr.groupby('City').agg({'Risk_Flag':'count'})
city_R = train_r.groupby('City').agg({'Risk_Flag':'count'})
df_city_NR = pd.DataFrame(data=city_NR).rename(columns={'Risk_Flag':'Count_NR'}).sort_values(by='Count_NR',ascending=False).reset_index()
df_city_R = pd.DataFrame(data=city_R).rename(columns={'Risk_Flag':'Count_R'}).sort_values(by='Count_R',ascending=False).reset_index()
df_city = df_city_NR.merge(df_city_R,left_on = 'City', right_on = 'City',how = 'right')
df_city['Total_Count'] = df_city.apply(lambda x : x['Count_R'] + x['Count_NR'],axis=1)
df_city['Ratio_R'] = df_city.apply(lambda x : x['Count_R'] / x['Total_Count'],axis=1)
df_city.sort_values(by='Ratio_R',ascending=False).reset_index().head(6)

,index,City,Count_NR,Count_R,Total_Count,Ratio_R
0,7,Bhubaneswar,409,198,607,0.326194
1,19,Gwalior,448,168,616,0.272727
2,76,Bettiah,335,122,457,0.266958
3,0,Kochi,718,243,961,0.252862
4,18,Raiganj,546,172,718,0.239554
5,87,Purnia,376,118,494,0.238866


In [17]:
# Mengecek urutan persentase rasio customer yang gagal membayar berdasarkan negara bagian
State_NR = train_nr.groupby('State').agg({'Risk_Flag':'count'})
State_R = train_r.groupby('State').agg({'Risk_Flag':'count'})
df_state_NR = pd.DataFrame(data=State_NR).rename(columns={'Risk_Flag':'Count_NR'}).sort_values(by='Count_NR',ascending=False).reset_index()
df_state_R = pd.DataFrame(data=State_R).rename(columns={'Risk_Flag':'Count_R'}).sort_values(by='Count_R',ascending=False).reset_index()
df_state = df_state_NR.merge(df_state_R,left_on = 'State', right_on = 'State',how = 'right')
df_state['Total_Count'] = df_state.apply(lambda x : x['Count_R'] + x['Count_NR'],axis=1)
df_state['Ratio_R'] = df_state.apply(lambda x : x['Count_R'] / x['Total_Count'],axis=1)
df_state.sort_values(by='Ratio_R',ascending=False).reset_index().head(6)

,index,State,Count_NR,Count_R,Total_Count,Ratio_R
0,20,Manipur,666,183,849,0.215548
1,22,Tripura,673,136,809,0.168109
2,13,Kerala,4835,970,5805,0.167097
3,19,Jammu_and_Kashmir,1497,283,1780,0.158989
4,5,Madhya_Pradesh,11942,2180,14122,0.154369
5,15,Odisha,3994,664,4658,0.142550


In [18]:
# Mengecek urutan persentase rasio customer yang gagal membayar berdasarkan pekerjaan
Prof_NR = train_nr.groupby('Profession').agg({'Risk_Flag':'count'})
Prof_R = train_r.groupby('Profession').agg({'Risk_Flag':'count'})
df_prof_NR = pd.DataFrame(data=Prof_NR).rename(columns={'Risk_Flag':'Count_NR'}).sort_values(by='Count_NR',ascending=False).reset_index()
df_prof_R = pd.DataFrame(data=Prof_R).rename(columns={'Risk_Flag':'Count_R'}).sort_values(by='Count_R',ascending=False).reset_index()
df_prof = df_prof_NR.merge(df_prof_R,left_on = 'Profession', right_on = 'Profession',how = 'right')
df_prof['Total_Count'] = df_prof.apply(lambda x : x['Count_R'] + x['Count_NR'],axis=1)
df_prof['Ratio_R'] = df_prof.apply(lambda x : x['Count_R'] / x['Total_Count'],axis=1)
df_prof.sort_values(by='Ratio_R',ascending=False).reset_index().head(6)

,index,Profession,Count_NR,Count_R,Total_Count,Ratio_R
0,0,Police_officer,4209,826,5035,0.164052
1,8,Chartered_Accountant,3803,690,4493,0.153572
2,5,Army_officer,3952,709,4661,0.152113
3,3,Surveyor,4000,714,4714,0.151464
4,1,Software_Developer,4303,750,5053,0.148427
5,9,Scientist,4091,690,4781,0.144321


In [19]:
# Berdasarkan urutan rasio customer dari ketiga kolom tersebut, maka dibuat encoding berdasarkan 5 peringkat teratas
# Konversi state menjadi kategori dari top 5 dan others [Manipur,Tripura,Kerala,Jammu Kashmir,Madya Pradesh,Others]
# Konversi profesi menjadi kategori dari top 5 dan others [Police,Accountant,Army,surveyor,software developer,Others]
# Kolom kota tidak dilakukan encoding karena sudah diwakili kolom state dan kolom state memiliki jumlah unique values lebih sedikit

# Konversi kategori state dan profession
list_cat_prof = []
list_cat_state = []

for i in train['State'] :
  if i == 'Manipur' :
    stat = 6
    list_cat_state.append(stat)
  elif i == 'Tripura' :
    stat = 5
    list_cat_state.append(stat)
  elif i == 'Kerala' :
    stat = 4
    list_cat_state.append(stat)
  elif i == 'Jammu_and_Kashmir' :
    stat = 3
    list_cat_state.append(stat)
  elif i == 'Madhya_Pradesh' :
    stat = 2
    list_cat_state.append(stat)
  else : 
    stat = 1
    list_cat_state.append(stat)

for i in train['Profession'] :
  if i == 'Police_officer' :
    prof = 6
    list_cat_prof.append(prof)
  elif i == 'Chartered_Accountant' :
    prof = 5
    list_cat_prof.append(prof)
  elif i == 'Army_officer' :
    prof = 4
    list_cat_prof.append(prof)
  elif i == 'Surveyor' :
    prof = 3
    list_cat_prof.append(prof)
  elif i == 'Software_Developer' :
    prof = 2
    list_cat_prof.append(prof)
  else : 
    prof = 1
    list_cat_prof.append(prof)

In [20]:
# Memasukan hasil encoding ke dataset
train['state_cat'] = list_cat_state
train['prof_cat'] = list_cat_prof
train

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,...,CURRENT_HOUSE_YRS,Risk_Flag,City,State,Experience_norm,Age_norm,Income_norm,House_norm,state_cat,prof_cat
0,1,1303834,23,3,0,1,0,Mechanical_engineer,Rewa,Madhya_Pradesh,...,13,0,Rewa,Madhya_Pradesh,0.15,0.034483,0.129487,0.75,2,1
1,2,7574516,40,10,0,1,0,Software_Developer,Parbhani,Maharashtra,...,13,0,Parbhani,Maharashtra,0.50,0.327586,0.757206,0.75,1,2
2,3,3991815,66,4,1,1,0,Technical_writer,Alappuzha,Kerala,...,10,0,Alappuzha,Kerala,0.20,0.775862,0.398564,0.00,4,1
3,4,6256451,41,2,0,1,1,Software_Developer,Bhubaneswar,Odisha,...,12,1,Bhubaneswar,Odisha,0.10,0.344828,0.625263,0.50,1,2
4,5,5768871,47,11,0,1,0,Civil_servant,Tiruchirappalli,Tamil_Nadu,...,14,1,Tiruchirappalli,Tamil_Nadu,0.55,0.448276,0.576454,1.00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,0,1,0,Surgeon,Kolkata,West_Bengal,...,11,0,Kolkata,West_Bengal,0.65,0.379310,0.815303,0.25,1,1
251996,251997,2843572,26,10,0,1,0,Army_officer,Rewa,Madhya_Pradesh,...,11,0,Rewa,Madhya_Pradesh,0.50,0.086207,0.283620,0.25,2,4
251997,251998,4522448,46,7,0,1,0,Design_Engineer,Kalyan-Dombivli,Maharashtra,...,12,0,Kalyan-Dombivli,Maharashtra,0.35,0.431034,0.451682,0.50,1,1
251998,251999,6507128,45,0,0,1,0,Graphic_Designer,Pondicherry,Puducherry,...,10,0,Pondicherry,Puducherry,0.00,0.413793,0.650356,0.00,1,1


F. Handle Class Imbalance

In [21]:
# Melihat proporsi data risk flag 1 dan 0
train['Risk_Flag'].value_counts()

0    221004
1     30996
Name: Risk_Flag, dtype: int64

Berdasarkan proporsi tersebut, diketahui bahwa terjadi class imbalance. Langkah terbaik untuk mengatasi hal tersebut adalah melakukan resample undersampling untuk bagian risk_flag=0 dan kemudian kelas risk _flag =1 dilakukan undersampling dengan SMOTE. 

Pengerjaan class imbalance pada risk flag akan dilakukan terakhir.

**2. Feature Engineering**

**A**. Feature Selection

Memilih kolom yang digunakan sebagai variabel penduga / membuang kolom yang kurang relevan atau redundan

Kolom/ variabel/ fitur yang tidak digunakan : kota/CITY, status(married/single), house_ownership, dan current_job_years tidak digunakan

*   Variabel kota tidak digunakan karena sudah diwakili oleh state dan variabel state memiliki nilai unik lebih sedikit
*   variabel status dan house ownership tidak digunakan karena perbandingan data sangat jauh (terlalu besar)

*   variabel current job years tidak digunakan karena bekorelasi cukup erat dengan experience sehingga dipilih satu saja (experience)

Bentuk model dibagi menjadi 2 skenario, yakni Regresi : logistik ; Klasifikasi : Decission tree dan KNeighbour Classification

B. Feature Extraction

In [22]:
# Mencari insight untuk membuat feature baru
# Apakah ada hubungan dari profiling user terhadap peluang gagal bayar?

profile = train.groupby(['Married/Single','House_Ownership','Car_Ownership']).agg({'Risk_Flag':'count'})
df_profile = pd.DataFrame(data=profile).reset_index()
df_profile

,Married/Single,House_Ownership,Car_Ownership,Risk_Flag
0,0,0,0,4602
1,0,0,1,1987
2,0,1,0,145032
3,0,1,1,62656
4,0,2,0,8355
5,0,2,1,3640
6,1,0,0,417
7,1,0,1,178
8,1,1,0,16986
9,1,1,1,7224


In [23]:
# Melihat nilai total customer tiap profile
total_customer = train.groupby(['Married/Single','House_Ownership']).agg({'House_Ownership':'count'}).rename(columns={'House_Ownership':'Total'})
df_total =  pd.DataFrame(data=total_customer).rename(columns={'House_Ownership':'Total'}).reset_index()
df_total

,Married/Single,House_Ownership,Total
0,0,0,6589
1,0,1,207688
2,0,2,11995
3,1,0,595
4,1,1,24210
5,1,2,923


In [24]:
# Memasukan kolom total dan menghitung rasio
conditions = [
    (df_profile['Married/Single'] == 0) & (df_profile['House_Ownership'] == 0),
    (df_profile['Married/Single'] == 0) & (df_profile['House_Ownership'] == 1),
    (df_profile['Married/Single'] == 0) & (df_profile['House_Ownership'] == 2),
    (df_profile['Married/Single'] == 1) & (df_profile['House_Ownership'] == 0),
    (df_profile['Married/Single'] == 1) & (df_profile['House_Ownership'] == 1),
    (df_profile['Married/Single'] == 1) & (df_profile['House_Ownership'] == 2),
]

#values = [6589,207688,11995,595,24210,923]
values = df_total['Total']

df_profile['Jumlah'] = np.select(conditions, values)
df_profile['Ratio'] = df_profile.apply(lambda x : x['Risk_Flag']/x['Jumlah'],axis=1)
df_profile

,Married/Single,House_Ownership,Car_Ownership,Risk_Flag,Jumlah,Ratio
0,0,0,0,4602,6589,0.698437
1,0,0,1,1987,6589,0.301563
2,0,1,0,145032,207688,0.698317
3,0,1,1,62656,207688,0.301683
4,0,2,0,8355,11995,0.696540
5,0,2,1,3640,11995,0.303460
6,1,0,0,417,595,0.700840
7,1,0,1,178,595,0.299160
8,1,1,0,16986,24210,0.701611
9,1,1,1,7224,24210,0.298389


Berdasarkan tabel dapat diketahui bahwa profile customer yang menikah, memiliki menyewa, dan memiliki mobil cenderung memiliki peluang resiko yang lebih besar daripada profil customer lainnya. (persentase yang gagall bayar naik sekitar 5% dibanding lainnya, yakni 34%)

Profile tersebut akan digunakan dalam fitur tambahan yakni married_own_carhouse.

In [25]:
# Feature Extraction
# Membuat fitur baru married_own_carhouse: bernilai 1 jika ya dan 0 jika tidak
train['married_own_carhouse'] = train.apply(lambda x : 1 if x['Married/Single']==1 and x['House_Ownership']==2 and x['Car_Ownership']==1 else 0,axis=1)

In [26]:
# Menambahkan feature class_income, age_class, exp_class

# Treshold yang digunakan berdasarkan interquartil data
# Pengurutan kode berdasarkan hasil urutan rasio dalam mencari insight

# Kode : low, medium, high
# Konversi kontinu ke kategorikal class_income : low(<2,5 jt) medium(2,5-7,5) high(>7,5 jt) 
# Konversi kontinu ke kategorikal class_age : low(<35) medium(35-65) high(>65) 
# Konversi kontinu ke kategorikal class_exp : low(<5) medium(5-15) high(>15) 

In [27]:
train['class_income'] = train.apply(lambda x : 'low' if x['Income']<2.503015e+06 else ('high' if x['Income']>7.477502e+06 else 'medium'),axis=1)
train['class_age'] = train.apply(lambda x : 'low' if x['Age']<35 else ('high' if x['Age']>65 else 'medium'),axis=1)
train['class_exp'] = train.apply(lambda x : 'low' if x['Experience']<5 else ('high' if x['Experience']>15 else 'medium'),axis=1)

In [28]:
# Mencari insight untuk membuat feature baru
# Apakah ada hubungan dari profiling user terhadap peluang gagal bayar berdasarkan masing-masing class income, class age, class exp?

In [29]:
# Melihat urutan rasio nilai total customer tiap profile class income
profile2 = train.groupby(['class_income']).agg({'Risk_Flag':'count'}).rename(columns={'Risk_Flag':'Jumlah'})
df_profile2 = pd.DataFrame(data=profile2).reset_index().rename(columns={'Risk_Flag':'Jumlah'})
total_customer2 = train.groupby(['class_income','Risk_Flag']).agg({'class_income':'count'}).rename(columns={'class_income':'Total'})
df_total2 =  pd.DataFrame(data=total_customer2).rename(columns={'class_income':'Total'}).reset_index().merge(df_profile2,left_on = 'class_income',right_on = 'class_income', how = 'left')
df_total2['Ratio'] = df_total2.apply(lambda x : x['Total']/x['Jumlah'],axis=1)
df_total2 

,class_income,Risk_Flag,Total,Jumlah,Ratio
0,high,0,55030,62988,0.873658
1,high,1,7958,62988,0.126342
2,low,0,54908,62967,0.872012
3,low,1,8059,62967,0.127988
4,medium,0,111066,126045,0.881161
5,medium,1,14979,126045,0.118839


In [30]:
# Melihat urutan rasio nilai total customer tiap profile class age
profile3 = train.groupby(['class_age']).agg({'Risk_Flag':'count'}).rename(columns={'Risk_Flag':'Jumlah'})
df_profile3 = pd.DataFrame(data=profile3).reset_index().rename(columns={'Risk_Flag':'Jumlah'})
total_customer3 = train.groupby(['class_age','Risk_Flag']).agg({'class_age':'count'}).rename(columns={'class_age':'Total'})
df_total3 =  pd.DataFrame(data=total_customer3).rename(columns={'class_age':'Total'}).reset_index().merge(df_profile3,left_on = 'class_age',right_on = 'class_age', how = 'left')
df_total3['Ratio'] = df_total3.apply(lambda x : x['Total']/x['Jumlah'],axis=1)
df_total3 

,class_age,Risk_Flag,Total,Jumlah,Ratio
0,high,0,53203,60223,0.883433
1,high,1,7020,60223,0.116567
2,low,0,52663,61083,0.862155
3,low,1,8420,61083,0.137845
4,medium,0,115138,130694,0.880974
5,medium,1,15556,130694,0.119026


In [31]:
# Melihat urutan rasio nilai total customer tiap profile class experience
profile4 = train.groupby(['class_exp']).agg({'Risk_Flag':'count'}).rename(columns={'Risk_Flag':'Jumlah'})
df_profile4 = pd.DataFrame(data=profile4).reset_index().rename(columns={'Risk_Flag':'Jumlah'})
total_customer4 = train.groupby(['class_exp','Risk_Flag']).agg({'class_exp':'count'}).rename(columns={'class_exp':'Total'})
df_total4 =  pd.DataFrame(data=total_customer4).rename(columns={'class_exp':'Total'}).reset_index().merge(df_profile4,left_on = 'class_exp',right_on = 'class_exp', how = 'left')
df_total4['Ratio'] = df_total4.apply(lambda x : x['Total']/x['Jumlah'],axis=1)
df_total4 

,class_exp,Risk_Flag,Total,Jumlah,Ratio
0,high,0,53852,60475,0.890484
1,high,1,6623,60475,0.109516
2,low,0,49272,57581,0.855699
3,low,1,8309,57581,0.144301
4,medium,0,117880,133944,0.880069
5,medium,1,16064,133944,0.119931


In [32]:
# Mengubah value kolom class income, class experience, dan class age menjadi int berdasarkan urutan rasionya (encoding)
# Semakin tinggi rasio gagal bayar maka nilai urutannya semakin besar

mapping_class_income = {
    'low' : 3,
    'medium' : 1,
    'high' : 2
}

mapping_class_age = {
    'low' : 3,
    'medium' : 2,
    'high' : 1
}

mapping_class_experience = {
    'low' : 3,
    'medium' : 2,
    'high' : 1
}

train['class_income'] = train['class_income'].map(mapping_class_income)
train['class_age'] = train['class_age'].map(mapping_class_age)
train['class_exp'] = train['class_exp'].map(mapping_class_experience)
train.head(5)

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,...,Experience_norm,Age_norm,Income_norm,House_norm,state_cat,prof_cat,married_own_carhouse,class_income,class_age,class_exp
0,1,1303834,23,3,0,1,0,Mechanical_engineer,Rewa,Madhya_Pradesh,...,0.15,0.034483,0.129487,0.75,2,1,0,3,3,3
1,2,7574516,40,10,0,1,0,Software_Developer,Parbhani,Maharashtra,...,0.50,0.327586,0.757206,0.75,1,2,0,2,2,2
2,3,3991815,66,4,1,1,0,Technical_writer,Alappuzha,Kerala,...,0.20,0.775862,0.398564,0.00,4,1,0,1,1,3
3,4,6256451,41,2,0,1,1,Software_Developer,Bhubaneswar,Odisha,...,0.10,0.344828,0.625263,0.50,1,2,0,1,2,3
4,5,5768871,47,11,0,1,0,Civil_servant,Tiruchirappalli,Tamil_Nadu,...,0.55,0.448276,0.576454,1.00,1,1,0,1,2,2


4 Feature tambahan yang telah dibuat :

1. married_house_car_ownership : melihat apakah customer memiliki ketiganya atau tidak.
2. class_exp : pengkategorian kelas pengalaman kerja dibagi berdasarkan kuartil.
3. class_income : pengkategorian kelas pendapatan dibagi berdasarkan kuartil.
4. class_age : pengkategorian kelas umur dibagi berdasarkan kuartil.

In [33]:
# Mempersiapkan data training model
# variabel income, experience, dan age dipilih bagian yg dinormalisasi karena lebih lebar jangkauannya

data_train = train[['Age_norm','Income_norm','Experience_norm','House_norm','Car_Ownership','state_cat','prof_cat','married_own_carhouse','Risk_Flag']]
data_train

,Age_norm,Income_norm,Experience_norm,House_norm,Car_Ownership,state_cat,prof_cat,married_own_carhouse,Risk_Flag
0,0.034483,0.129487,0.15,0.75,0,2,1,0,0
1,0.327586,0.757206,0.50,0.75,0,1,2,0,0
2,0.775862,0.398564,0.20,0.00,0,4,1,0,0
3,0.344828,0.625263,0.10,0.50,1,1,2,0,1
4,0.448276,0.576454,0.55,1.00,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...
251995,0.379310,0.815303,0.65,0.25,0,1,1,0,0
251996,0.086207,0.283620,0.50,0.25,0,2,4,0,0
251997,0.431034,0.451682,0.35,0.50,0,1,1,0,0
251998,0.413793,0.650356,0.00,0.00,0,1,1,0,0


In [34]:
# Deklarasi nilai x dan y
x = data_train[['Age_norm','Income_norm','Experience_norm','House_norm','Car_Ownership','state_cat','prof_cat','married_own_carhouse']]
y = data_train[['Risk_Flag']]

In [35]:
# Handling class imbalance menggunakan pipeline SMOTE dan undersampling
over = SMOTE(sampling_strategy=0.3, random_state=4)
under = RandomUnderSampler(sampling_strategy=0.3, random_state=4)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
pipeline

Pipeline(steps=[('o', SMOTE(random_state=4, sampling_strategy=0.3)),
                ('u',
                 RandomUnderSampler(random_state=4, sampling_strategy=0.3))])

In [36]:
# Handling class imbalance menggunakan SMOTE dan undersampling
x_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(sampling_strategy=0.3,random_state=4).fit_resample(x,y)
x_under, y_under = under_sampling.RandomUnderSampler(0.3,random_state=4).fit_resample(x_over_SMOTE,y_over_SMOTE)

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.3 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


In [37]:
y_under.value_counts()

Risk_Flag
0            221003
1             66301
dtype: int64

**FEATURE CHECKING**

In [38]:
# Data asli
['Id', 'Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS',  ]

['Id',
 'Married/Single',
 'House_Ownership',
 'Car_Ownership',
 'Profession',
 'CITY',
 'STATE',
 'CURRENT_JOB_YRS',
 'CURRENT_HOUSE_YRS']

In [39]:
status_risk_flag = pd.crosstab(train["Married/Single"], train["Risk_Flag"])
status_risk_flag

Risk_Flag,0,1
Married/Single,,
0,197912,28360
1,23092,2636


In [40]:
car_ownership_risk_flag = pd.crosstab(train["Car_Ownership"], train["Risk_Flag"])
car_ownership_risk_flag

Risk_Flag,0,1
Car_Ownership,,
0,153439,22561
1,67565,8435


In [41]:
house_ownership_risk_flag = pd.crosstab(train["House_Ownership"], train["Risk_Flag"])
house_ownership_risk_flag

Risk_Flag,0,1
House_Ownership,,
0,6469,715
1,202777,29121
2,11758,1160


In [42]:
class_age_risk_flag = pd.crosstab(train["class_age"], train["Risk_Flag"])
class_age_risk_flag

Risk_Flag,0,1
class_age,,
1,53203,7020
2,115138,15556
3,52663,8420


In [43]:
class_income_risk_flag = pd.crosstab(train["class_income"], train["Risk_Flag"])
class_income_risk_flag

Risk_Flag,0,1
class_income,,
1,111066,14979
2,55030,7958
3,54908,8059


In [44]:
class_exp_risk_flag = pd.crosstab(train["class_exp"], train["Risk_Flag"])
class_exp_risk_flag

Risk_Flag,0,1
class_exp,,
1,53852,6623
2,117880,16064
3,49272,8309


In [48]:
# Export data ke csv (download)

data_train.to_csv('/content/drive/MyDrive/Source Code/Data_Train_Model.csv')  